<a href="https://colab.research.google.com/github/davidlealo/100profes/blob/master/modelo_transcripcion_segmentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalar dependencias necesarias
!pip install openai-whisper pyannote.audio spacy python-dotenv
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Importar librerías
from pyannote.audio import Pipeline
from whisper import load_model
import spacy
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Cargar modelos
whisper_model = load_model("base")
nlp = spacy.load("en_core_web_sm")

# Descargar el pipeline de diarización
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization",
    use_auth_token=HUGGINGFACE_TOKEN
)

# Función para procesar un archivo de audio
def process_audio(file_path):
    # Diarización
    diarization = diarization_pipeline(file_path)
    person_mapping = {}

    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start, end = int(turn.start * 1000), int(turn.end * 1000)
        text = whisper_model.transcribe(file_path, task="transcribe", segment=(start, end))['text']

        # Analizar texto para detectar nombres
        doc = nlp(text)
        names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

        if names:
            person_mapping[speaker] = names[0]

    return {"person_mapping": person_mapping}

# Subir archivo de audio
from google.colab import files
uploaded = files.upload()

# Procesar archivo subido
for file_name in uploaded.keys():
    result = process_audio(file_name)
    print(result)


config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]


Could not download 'pyannote/segmentation' model.
It might be because the model is private or gated so make
sure to authenticate. Visit https://hf.co/settings/tokens to
create your access token and retry with:

   >>> Model.from_pretrained('pyannote/segmentation',
   ...                       use_auth_token=YOUR_AUTH_TOKEN)

If this still does not work, it might be because the model is gated:
visit https://hf.co/pyannote/segmentation to accept the user conditions.


AttributeError: 'NoneType' object has no attribute 'eval'